<a href="https://colab.research.google.com/github/mctavares/AI/blob/GenAISQLAgent/Copy_of_GenAISQLAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Gen AI SQL Agent
Ih this notebook we create a simple SQL Agent using Hugging Face.

In [ ]:
# Check to see if dependencies are already installed
import transformers
import torch
import sqlalchemy
import transformers.agents

print('Hugging Face Transformers version:' + transformers.__version__)
print('PyTorch version:' + transformers.__version__)
print('sqlalchemy version:' + sqlalchemy.__version__)

This block only needs to be run once. Update it as needed if things change and there are errors.

In [ ]:
# Add here list of depedencies thta need to be installed
# !pip install sqlalchemy
!pip install fsspec
!pip install --upgrade transformers
!pip install transformers[agents]
!pip install --upgrade sqlalchemy

For this exercise we are going to create an in memory sqlite database. The drawback is that, once we are done, the data is gone. However, if we want to persist data in the future, we just need to remove the comment as indicated in the code.

There are two tables:
country: contains names of some european countries
league: contains names of european football leagues and references to their countries.

Later on we will be asking questions about both the countries and the leagues.

In [27]:
# Let's create a simple database
import sqlalchemy as db

# Based on https://www.kaggle.com/code/alaasedeeq/european-soccer-database-with-sqlite3
# engine = db.create_engine("sqlite:///european_database.sqlite") # Uncomment if want db in disk

metadata = db.MetaData()

# Country table
country = db.Table('country', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False)
              )
# League table
league = db.Table('league', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Country_Id', db.Integer(),db.ForeignKey('country.Id')),
              db.Column('Name', db.String(255), nullable=False)
              )
engine = db.create_engine('sqlite:///:memory:') #, echo=True) # Comment out for verbose creation
metadata.create_all(engine)

# Connection will be open until we execute the agent since db is inn memory
conn = engine.connect()

Let's populate the country table with some data.

In [28]:
insert_stmt = country.insert().values([
{'Id':1, 'Name':'Belgium'},
{'Id':1729, 'Name':'England'},
{'Id':4769, 'Name':'France'},
{'Id':7809, 'Name':'Germany'},
{'Id':10257, 'Name':'Italy'},
{'Id':13274, 'Name':'Netherlands'},
{'Id':15722, 'Name':'Poland'},
{'Id':17642, 'Name':'Portugal'},
{'Id':19694, 'Name':'Scotland'},
{'Id':21518, 'Name':'Spain'},
{'Id':24558, 'Name':'Switzerland'}
])

conn.execute(insert_stmt)
output = conn.execute(country.select()).fetchall()

Now we create some data for the leagues table. Notice that we need the coutry table first because of the foreign keys.

In [29]:
insert_stmt = league.insert().values([
{'Id':1,'Country_Id':1,'Name':'Belgium Jupiler League'},
{'Id':1729,'Country_Id':1729,'Name':'England Premier League'},
{'Id':4769,'Country_Id':4769,'Name':'France Ligue 1'},
{'Id':7809,'Country_Id':7809,'Name':'Germany 1. Bundesliga'},
{'Id':10257,'Country_Id':10257,'Name':'Italy Serie A'},
{'Id':13274,'Country_Id':13274,'Name':'Netherlands Eredivisie'},
{'Id':15722,'Country_Id':15722,'Name':'Poland Ekstraklasa'},
{'Id':17642,'Country_Id':17642,'Name':'Portugal Liga ZON Sagres'},
{'Id':19694,'Country_Id':19694,'Name':'Scotland Premier League'},
{'Id':21518,'Country_Id':21518,'Name':'Spain LIGA BBVA	21518'},
{'Id':24558,'Country_Id':24558,'Name':'Switzerland Super League'}
])

conn.execute(insert_stmt)
output = conn.execute(league.select()).fetchall()

The SQL Agent code is straightforward. We define a function (tool) that will execute the query and return the results. The LLM receives the request (input) from the user and translates the request into a SQL command, which is then passed to the query_tool function. The query_funtion executes the query and sends the results back to the LLM, which then transforms the results into a format a human can understand.

Hugging Face requires that you describe what the query_tool function does using pyhon comments, as well as the schema of each table that can be queried.

In [30]:
# Hugging Face code
from transformers import ReactCodeAgent, HfApiEngine, Tool, tool

@tool
def query_tool(query: str) -> str:
  """
  Allows you to perform SQL queries on the table. Returns a string representation of the result.

  It can use the tables 'country' and 'league'.

  The table 'country' has the following columns:
  Columns:
  - Id: INTEGER
  - Name: VARCHAR(16)

  The table 'league' has the following columns:
  Columns:
  - Id: INTEGER
  - Country_Id: INTEGER
  - Name: VARCHAR(16)

  Args:
  query: The query to perform. This should be correct SQL.
  """
  output = ""
  rows = conn.execute(db.text(query))
  for row in rows:
    output += "\n" + str(row)
  return output


agent = ReactCodeAgent(
    tools=[query_tool],
    llm_engine=HfApiEngine("meta-llama/Meta-Llama-3-8B-Instruct")
)

Now we run the agents. We want to test the following:
```
# Find country/countries that start with the letter p
```
This will query the country table only.


In [ ]:
agent.run("Can you give me the name of a country that starts with the letter p?")

The next request is:
```
# Can you give me the name of the league associated with the country Italy?
```
This will perform a join between the league and country tables.

In [ ]:
agent.run("Can you give me the name of the league associated with the country Italy?")

Finally we ask for something that should not be in the table:
```
# Is the country Ireland in the list of countries?
```
This will scan the country tables and find nothing

In [ ]:
agent.run("Is the country Ireland in the list of countries?")

Once done, we close the connection and the data is gone.

In [ ]:
conn.close()
print ('Done')

And that's it! We wrote a simple SQL Agent that can answer questions by querying two related tables.

# References
## Data source:
* https://www.kaggle.com/code/dimarudov/data-analysis-using-sql

## SQLAlchemy:
* https://www.datacamp.com/tutorial/sqlalchemy-tutorial-examples
* https://www.slingacademy.com/article/sqlalchemy-how-to-bulk-insert-data-into-a-table/
* https://docs.sqlalchemy.org/en/20/orm/join_conditions.html
* https://www.sqlitetutorial.net/sqlite-go/create-table/

## Hugging Face
* https://huggingface.co/learn/cookbook/en/agent_text_to_sql